In [13]:
library(rio)
library(reshape2)
library(stringr)
library(ggthemes)
library(dplyr)
library(haven)
library(descr)
library(car)
library(ggplot2)
library(ggpmisc)
library(gridExtra)
library(haven)
library(poLCA)

In [14]:
d_20 = import("2020.sav")
d_23 = import("2023.sav")

Invalid date string (length=9): 06 028 21
Invalid date string (length=9): 09 034 23


In [15]:
# Definir las variables que quieres revisar para cada dataframe
variables_2020 <- c("IDENPA", "NUMINVES", "P22STM.B", "P21STM", "P20STM.D", 
                     "P13STGBS.A", "P13STGBS.B", "P13ST.C", "P13ST.D", 
                     "P13ST.E", "P13ST.F", "P13ST.G", "P13ST.I", 
                     "P13ST.H", "P18ST", "P20ST.A", "P11STGBS.A", 
                     "P4STGBS", "SEXO", "REEDUC.1", "P46STGBS", "P3STGBS")

variables_2023 <- c("IDENPA", "NUMINVES", "P18STM.B", "P20STM", "P18STM.C", 
                     "P13STGBS.A", "P13STGBS.B", "P13ST.C", "P13ST.D", 
                     "P13ST.E", "P13ST.F", "P13ST.G", "P13ST.I", 
                     "P13ST.H", "P16ST", "P18ST.A", "P11STGBS.A", 
                     "P5STGBS", "SEXO", "REEEDUC.1", "P40STGBS","P4STGBS")

# Función para verificar si las variables existen
check_variables <- function(data, variables) {
  existing_vars <- intersect(names(data), variables)
  missing_vars <- setdiff(variables, existing_vars)
  
  return(list(existing = existing_vars, missing = missing_vars))
}

# Comprobar las variables para d_20 y d_23
result_2020 <- check_variables(d_20, variables_2020)
result_2023 <- check_variables(d_23, variables_2023)

# Mostrar resultados
cat("Variables en d_20:\n")
cat("Existen:", paste(result_2020$existing, collapse = ", "), "\n")
cat("No existen:", paste(result_2020$missing, collapse = ", "), "\n\n")

cat("Variables en d_23:\n")
cat("Existen:", paste(result_2023$existing, collapse = ", "), "\n")
cat("No existen:", paste(result_2023$missing, collapse = ", "), "\n")


Variables en d_20:
Existen: NUMINVES, IDENPA, SEXO, P3STGBS, P4STGBS, P11STGBS.A, P13STGBS.A, P13STGBS.B, P13ST.C, P13ST.D, P13ST.E, P13ST.F, P13ST.G, P13ST.H, P13ST.I, P18ST, P20ST.A, P22STM.B, P20STM.D, P21STM, P46STGBS, REEDUC.1 
No existen:  

Variables en d_23:
Existen: NUMINVES, IDENPA, SEXO, P4STGBS, P5STGBS, P11STGBS.A, P13STGBS.A, P13STGBS.B, P13ST.C, P13ST.D, P13ST.E, P13ST.F, P13ST.G, P13ST.H, P13ST.I, P16ST, P18ST.A, P18STM.B, P18STM.C, P20STM, P40STGBS, REEEDUC.1 
No existen:  


In [16]:
# Función para filtrar las variables de un data frame
filter_variables <- function(data, variables) {
  # Verificar si todas las variables existen
  existing_vars <- intersect(names(data), variables)
  
  # Retornar el data frame con solo las variables existentes
  return(data[, existing_vars, drop = FALSE])
}

# Filtrar las variables para d_20 y d_23
d_20_filtered <- filter_variables(d_20, variables_2020)
d_23_filtered <- filter_variables(d_23, variables_2023)

# Mostrar los data frames filtrados
print("Data frame d_20 filtrado:")
print(head(d_20_filtered))

print("Data frame d_23 filtrado:")
print(head(d_23_filtered))


[1] "Data frame d_20 filtrado:"
  NUMINVES IDENPA SEXO P3STGBS P4STGBS P11STGBS.A P13STGBS.A P13STGBS.B P13ST.C
1     2020     32    2      96       5          4          2          4       4
2     2020     32    1      96       5          4          1          1       3
3     2020     32    1       9       5          4          1          2       1
4     2020     32    2       9       5          4          1          2       2
5     2020     32    1       6       4          3          2          3       1
6     2020     32    2      45       3          2          2          2       1
  P13ST.D P13ST.E P13ST.F P13ST.G P13ST.H P13ST.I P18ST P20ST.A P22STM.B
1       4       4       4       4       3       4     5       2        2
2       2       4       2       4       2       4     5       1        4
3       2       1       2       2       2       1     5       1        2
4       3       4       4       4       4       3    10       3        1
5       2       2       2       2       2  

In [17]:
colnames(d_23_filtered)

[1] "NUMINVES"   "IDENPA"     "SEXO"       "P4STGBS"    "P5STGBS"   
 [6] "P11STGBS.A" "P13STGBS.A" "P13STGBS.B" "P13ST.C"    "P13ST.D"   
[11] "P13ST.E"    "P13ST.F"    "P13ST.G"    "P13ST.H"    "P13ST.I"   
[16] "P16ST"      "P18ST.A"    "P18STM.B"   "P18STM.C"   "P20STM"    
[21] "P40STGBS"   "REEEDUC.1"

In [18]:
colnames(d_20_filtered)

[1] "NUMINVES"   "IDENPA"     "SEXO"       "P3STGBS"    "P4STGBS"   
 [6] "P11STGBS.A" "P13STGBS.A" "P13STGBS.B" "P13ST.C"    "P13ST.D"   
[11] "P13ST.E"    "P13ST.F"    "P13ST.G"    "P13ST.H"    "P13ST.I"   
[16] "P18ST"      "P20ST.A"    "P22STM.B"   "P20STM.D"   "P21STM"    
[21] "P46STGBS"   "REEDUC.1"

In [19]:
library(dplyr)

d_20_filtered <- d_20_filtered %>%
  rename(
    Año = NUMINVES,
    Pais = IDENPA,
    SEXO = SEXO,
    situ_eco = P4STGBS,
    satisfaccion = `P11STGBS.A`,
    confianza_ffa = `P13STGBS.A`,
    confianza_polis = `P13STGBS.B`,
    confianza_iglesia = `P13ST.C`,
    confianza_congreso = `P13ST.D`,
    confianza_gob = `P13ST.E`,
    confianza_pj = `P13ST.F`,
    confianza_partidos = `P13ST.G`,
    confianza_orga_elector = `P13ST.H`,
    confianza_presi = `P13ST.I`,
    izq_der = `P18ST`,
    democracia_mejor = `P20ST.A`,
    autoritario_problemas = `P22STM.B`,
    control_medios = `P20STM.D`,
    gob_militar = `P21STM`,
    interes = P46STGBS,
    Educacion = REEDUC.1
  )


In [20]:
d_23_filtered <- d_23_filtered %>%
  rename(
    Año = NUMINVES,
    Pais = IDENPA,
    SEXO = SEXO,
    situ_eco = P5STGBS,
    satisfaccion = `P11STGBS.A`,
    confianza_ffa = `P13STGBS.A`,
    confianza_polis = `P13STGBS.B`,
    confianza_iglesia = `P13ST.C`,
    confianza_congreso = `P13ST.D`,
    confianza_gob = `P13ST.E`,
    confianza_pj = `P13ST.F`,
    confianza_partidos = `P13ST.G`,
    confianza_orga_elector = `P13ST.H`,
    confianza_presi = `P13ST.I`,
    izq_der = P16ST,
    democracia_mejor = P18ST.A,
    autoritario_problemas = P18STM.B,
    control_medios = P18STM.C,
    gob_militar = P20STM,
    interes = P40STGBS,
    Educacion = REEEDUC.1
  )


In [21]:
df_combined <- full_join(d_20_filtered, d_23_filtered, by = c("Año", "Pais", "SEXO", "situ_eco", "satisfaccion", 
                                                             "confianza_ffa", "confianza_polis", "confianza_iglesia",
                                                             "confianza_congreso", "confianza_gob", "confianza_pj",
                                                             "confianza_partidos", "confianza_orga_elector",
                                                             "confianza_presi", "izq_der", "democracia_mejor",
                                                             "autoritario_problemas", "control_medios", 
                                                             "gob_militar", "interes", "Educacion","problema","apoyo"))


ERROR: [1m[33mError[39m in `full_join()`:[22m
[1m[22m[33m![39m Join columns in `x` must be present in the data.
[31m✖[39m Problem with `democracia_mejor`.


In [10]:
df_combined

Año,Pais,SEXO,problema,situ_eco,satisfaccion,confianza_ffa,confianza_polis,confianza_iglesia,confianza_congreso,⋯,confianza_partidos,confianza_orga_elector,confianza_presi,izq_der,democracia_mejor,autoritario_problemas,control_medios,gob_militar,interes,Educacion
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2020,32,2,96,5,4,2,4,4,4,⋯,4,3,4,5,2,2,4,1,4,2
2020,32,1,96,5,4,1,1,3,2,⋯,4,2,4,5,1,4,4,1,1,6
2020,32,1,9,5,4,1,2,1,2,⋯,2,2,1,5,1,2,2,1,2,2
2020,32,2,9,5,4,1,2,2,3,⋯,4,4,3,10,3,1,2,1,4,3
2020,32,1,6,4,3,2,3,1,2,⋯,2,2,2,97,3,3,2,2,2,3
2020,32,2,45,3,2,2,2,1,3,⋯,4,2,2,97,1,3,3,NA,4,3
2020,32,2,9,5,3,2,3,2,4,⋯,4,4,3,97,3,3,4,2,3,7
2020,32,1,9,4,3,4,4,4,3,⋯,4,3,3,5,3,2,2,2,3,4
2020,32,1,35,5,3,2,2,4,3,⋯,4,2,4,5,2,3,4,2,1,7


In [11]:
df_combined <- df_combined %>%
  # Convertir 23 en 2023 en la columna 'NUMINVES'
  mutate(Año= ifelse(Año== 23, 2023, Año)) %>%
  # Filtrar para quedarte solo con los IDENPA de interés
  filter(Pais %in% c(604, 218, 170, 68)) %>%
  # Renombrar los valores de IDENPA
  mutate(Pais = case_when(
    Pais == 604 ~ "Perú",
    Pais == 218 ~ "Ecuador",
    Pais == 170 ~ "Colombia",
    Pais == 68 ~ "Bolivia",
    TRUE ~ as.character(Pais)  # Asegurarte de mantener otros valores (si los hubiera)
  ))

# Ver las primeras filas del dataframe modificado
head(df_combined)

,Año,Pais,SEXO,problema,situ_eco,satisfaccion,confianza_ffa,confianza_polis,confianza_iglesia,confianza_congreso,⋯,confianza_partidos,confianza_orga_elector,confianza_presi,izq_der,democracia_mejor,autoritario_problemas,control_medios,gob_militar,interes,Educacion
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2020,Bolivia,1,13,3,3,4,4,1,3,⋯,4,4,4,7,2,2,2,2,4,2
2,2020,Bolivia,1,34,3,4,2,3,1,3,⋯,4,4,3,6,2,2,3,1,3,5
3,2020,Bolivia,2,20,3,3,3,2,1,3,⋯,4,3,4,10,2,2,3,1,2,2
4,2020,Bolivia,1,35,4,4,2,3,1,2,⋯,3,3,3,8,3,2,4,2,3,5
5,2020,Bolivia,1,2,3,3,1,2,1,3,⋯,4,4,3,5,2,2,3,1,3,7
6,2020,Bolivia,1,9,4,3,3,3,2,2,⋯,3,3,3,7,2,2,3,1,4,5


In [12]:
write.csv(df_combined, "C:/Users/Nicolas Silva/Documents/GitHub/Tesis/Main_data/main_data1.csv", row.names=FALSE)
